In [1]:
import polars as pl
import pandas as pd

In [2]:
tmp = pl.scan_csv('data/CUSTOMER_SCORE.csv')
#tmp_2 = pl.scan_csv('data/CUSTOMER_누락제외.csv') 

In [4]:
tmp.collect_schema().names()

['기준년월', '고객ID', '연령대', '성별', '고객등급', '입출금예금', '기타예금', '비이자상품', '카드', '대출']

In [5]:
def calculate_financial_scores_v2(lazy_df):
    """
    금융 데이터에 대해 3개월 평균, 3개월 누적, 6개월 평균을 기반으로 점수를 계산하는 함수.
    LazyFrame을 입력받아 모든 연산을 지연 평가로 처리합니다.
    """
    # 단위 점수 기준
    unit_scores = {
        "입출금예금": 8 / 100000,
        "기타예금": 4 / 100000,
        "비이자상품": 6 /100000,
        "카드": 6 / 100000,
        "대출": 25 / 1000
    }
    
    # 모든 연산을 LazyFrame 체인으로 구성
    return (lazy_df
            # 3개월 평균/누적, 6개월 평균 계산
            .with_columns([
                pl.col("입출금예금").rolling_mean(window_size=3).alias("입출금예금_3개월평균"),
                pl.col("기타예금").rolling_mean(window_size=3).alias("기타예금_3개월평균"),
                pl.col("비이자상품").rolling_mean(window_size=3).alias("비이자상품_3개월평균"),
                pl.col("카드").rolling_sum(window_size=3).alias("카드_3개월누적"),
                pl.col("대출").rolling_mean(window_size=6).alias("대출_6개월평균")
            ])
            # 점수 계산
            .with_columns([
                (pl.col("입출금예금_3개월평균") * unit_scores["입출금예금"]).alias("입출금예금_점수"),
                (pl.col("기타예금_3개월평균") * unit_scores["기타예금"]).alias("기타예금_점수"),
                (pl.col("비이자상품_3개월평균") * unit_scores["비이자상품"]).alias("비이자상품_점수"),
                (pl.col("카드_3개월누적") * unit_scores["카드"]).alias("카드_점수"),
                (pl.col("대출_6개월평균") * 0.1 * unit_scores["대출"]).alias("대출_점수")
            ])
            # 총점수 계산
            .with_columns([
                (pl.col("입출금예금_점수") + 
                 pl.col("기타예금_점수") + 
                 pl.col("비이자상품_점수") + 
                 pl.col("카드_점수") + 
                 pl.col("대출_점수")).alias("총점수")
            ]))

# 2. 함수 적용 (여전히 LazyFrame 상태 유지)
result_lazy = calculate_financial_scores_v2(tmp_2)

# 3. 필요한 시점에 collect 호출
#final_df = result_lazy.collect()

In [6]:
#final_df.get_column('총점수').hist()

breakpoint,category,count
f64,cat,u32
24208.333333,"""(-242.083333, 24208.333333]""",60325738
48416.666667,"""(24208.333333, 48416.666667]""",4054879
72625.0,"""(48416.666667, 72625.0]""",382243
96833.333333,"""(72625.0, 96833.333333]""",61702
121041.666667,"""(96833.333333, 121041.666667]""",17964
145250.0,"""(121041.666667, 145250.0]""",8268
169458.333333,"""(145250.0, 169458.333333]""",3943
193666.666667,"""(169458.333333, 193666.666667]""",1777
217875.0,"""(193666.666667, 217875.0]""",718


In [7]:
'''

# 현재 최대값 확인
current_max = final_df.get_column('총점수').max()

# 스케일링 적용
scaled_df = final_df.with_columns([
    (pl.col('총점수') * (30000 / current_max)).alias('총점수_scaled')
])

# 결과 확인
print("=== 스케일링 결과 확인 ===")
print("새로운 최대값:", scaled_df.get_column('총점수_scaled').max())
print("\n=== 스케일링된 총점수 기본 통계량 ===")
print(scaled_df.get_column('총점수_scaled').describe())

'''

=== 스케일링 결과 확인 ===
새로운 최대값: 30000.0

=== 스케일링된 총점수 기본 통계량 ===
shape: (9, 2)
┌────────────┬─────────────┐
│ statistic  ┆ value       │
│ ---        ┆ ---         │
│ str        ┆ f64         │
╞════════════╪═════════════╡
│ count      ┆ 6.4857319e7 │
│ null_count ┆ 5.0         │
│ mean       ┆ 684.67085   │
│ std        ┆ 1306.88045  │
│ min        ┆ 0.0         │
│ 25%        ┆ 19.034768   │
│ 50%        ┆ 86.251291   │
│ 75%        ┆ 732.309811  │
│ max        ┆ 30000.0     │
└────────────┴─────────────┘


# Data analyzing


In [3]:
tmp_collected = tmp.filter([
    (pl.col('연령대') == 20) |
    (pl.col('연령대') == 30)
]).collect()

# lazy로 처리?

In [4]:
pandas_df = tmp_collected.to_pandas()

In [5]:
pandas_df.to_csv('data/output_2030.csv', index=False)